In [ ]:
import pandas as pd
dataTrain = pd.read_csv("tsv/train.tsv", sep='\t')

dataValid = pd.read_csv("tsv/validation.tsv", sep='\t')
dataTest = pd.read_csv("tsv/test.tsv", sep='\t')

In [ ]:
import math
total = len(dataTrain)
batch_size = math.floor(total * 10/100)

max_batch_i = math.floor(total/batch_size)
# batch_df = train_valid_df[batch_i*batch_size:(batch_i+1)*batch_size]
print(f'total: {total}, batch_size: {batch_size}, max_batch_i: {max_batch_i}')


batch_i = 9 # 0 - 9
if batch_i == max_batch_i - 1:
    batch_df = dataTrain[batch_i*batch_size:]
else :
    batch_df = dataTrain[batch_i*batch_size:(batch_i+1)*batch_size]
print(len(batch_df))


In [ ]:
# batch_df = dataTest

In [ ]:
batch_df.shape

### Run this 2 times cos some files fails to download on the first run

In [ ]:
import pandas as pd
import requests
import os
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor

def download_file(url, folder_path, file_name):
    file_name = file_name.replace('.wav', '').replace('.mp3', '')

    ext = url.split('.')[-1]
    file_name = os.path.join(folder_path, f"{file_name}.{ext}")

    # Skip downloading if the file already exists
    if os.path.exists(file_name):
        # print(f"File already exists: {file_name}")
        return
    
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=128):
                    f.write(chunk)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Failed to download from {url}")
    except Exception as e:
        print(f"Error occurred for {url}: {str(e)}")

def download_audio_files(df, folder_path, max_workers=5):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for index, row in df.iterrows():
            executor.submit(download_file, row['url'], folder_path, row['file_name'])

download_audio_files(batch_df, '/media/monlamai/SSD/wav2vec2/segments')

In [ ]:
batch_df[batch_df['file_name'] == 'STT_TT00234_00966.800-00969.300'].to_csv('test.csv', index=False)

## Convert files to wav

In [ ]:
import os
import magic
from tqdm.auto import tqdm
from pydub import AudioSegment

def resample_and_convert(df, folder_path, target_sample_rate=16000):
    # Initialize the magic library
    mime = magic.Magic(mime=True)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        filename = row['file_name'].replace('.mp3', '').replace('.wav', '') + '.' +row['url'].split('.')[-1]

        file_path = os.path.join(folder_path, filename)
        file_mime_type = mime.from_file(file_path)
        new_filename = filename.replace('.wav','').replace('.mp3','') + '.wav'
        new_file_path = os.path.join(folder_path, new_filename)

        if 'audio' in file_mime_type:
            # Process audio files
            # print(f"Processing {filename}...")
            audio = AudioSegment.from_file(file_path)

            if 'mpeg' in file_mime_type or audio.frame_rate != target_sample_rate or filename.endswith('.mp3'):
                # Convert to wav and resample if it's mp3 or sample rate is different
                resampled_audio = audio.set_frame_rate(target_sample_rate)
                resampled_audio.export(new_file_path, format='wav')
                print(f"{filename} has been converted and resampled to {new_filename}.")

resample_and_convert(batch_df, '/media/monlamai/SSD/wav2vec2/segments/')

## Confirm if the files are correct

In [ ]:
import os
import magic
from pydub import AudioSegment
from tqdm.auto import tqdm


segments = '/media/monlamai/SSD/wav2vec2/segments/'
mime = magic.Magic(mime=True)
for index, row in tqdm(batch_df.iterrows(), total=len(batch_df)):
    wav_file_name = row['file_name'].replace('.mp3', '').replace('.wav', '') + '.wav'
    if not os.path.exists(f'{segments}{wav_file_name}'):
        print(f'{wav_file_name} not found')
        continue
    file_mime_type = mime.from_file(f'{segments}{wav_file_name}')
    if 'mpeg' in file_mime_type:
        print(f'{wav_file_name} is mpeg')
        continue
    audio = AudioSegment.from_file(f'{segments}{wav_file_name}')
    if audio.frame_rate != 16000:
        print(f'{wav_file_name} is not 16000')
        continue
        

In [ ]:
batch_df['path'] = batch_df['file_name'].apply(lambda x: f'/media/monlamai/SSD/wav2vec2/segments/{x.replace(".wav","").replace(".mp3","")}.wav')

In [ ]:
import os
batch_df['path'].apply(lambda x: os.path.isfile(x)).value_counts()